In [1]:
import duckdb

In [9]:
duckdb.sql("""

    SELECT count(*) 
    FROM '../db/VCF_annovar/entete_variant.parquet' AS ENTETE

""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       274999 │
└──────────────┘

In [16]:
duckdb.sql("""
    SELECT * 
    FROM '../db/VCF_annovar/entete_variant.parquet' AS ENTETE 
    LEFT JOIN '../db/VCF_annovar/sample_variant.parquet' AS SAMPLE ON ENTETE.HASH = SAMPLE.HASH
    LEFT JOIN '../db/VCF_annovar/info_variant.parquet' AS INFO ON ENTETE.HASH = INFO.HASH
    LEFT JOIN '../db/VCF_annovar/info_variant.parquet' AS INFO2 ON ENTETE.HASH = INFO2.HASH
    ORDER BY ENTETE.HASH
""")

┌──────────────────────┬──────────────────┬───────────┬─────────────┬─────────┬─────────┬─────────┬────────────┬──────────────────────┬──────────────────────┬──────────────────────┬─────────┬──────────────────────┬──────────────────────┬──────────────────────┬─────────┬─────────┬─────────┬──────────────┬─────────┬───────────┬─────────┬─────────┬─────────┬─────────┬───────────┬─────────┬────────────────┬─────────┬──────────────┬─────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬─────────────┬─────────────┬─────────────────┬──────────────────┬───────────────────┬───────────────────┬───────────────────┬───────────────────┬───────────────────┬───────────────────┬───────────────────┬───────────────────┬──────────────────┬──────────────────┬──────────────────┬──────────────────┬──────────────────┬──────────────────┬──────────────────┬──────────────────┬───────────────────┬───────────────────┬───────────────────┬──────────────────